<a href="https://colab.research.google.com/github/kktsuji/train-lora-cell-rois/blob/main/generate_cell_rois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from peft import PeftModel

In [ ]:
# Variable Settings

# RESOLUTION = "40" # original size
# RESOLUTION = "256"
RESOLUTION = "512" # recommended for sd-1.x

NUM_TRAIN_EPOCHS = "200"

lora_dir_name = f"cell-rois_lora_resolution-{RESOLUTION}_epochs-{NUM_TRAIN_EPOCHS}"

In [ ]:
import os

lora_path = f"/content/drive/MyDrive/Research/LoRAs/{lora_dir_name}/unet/"
print(lora_path)

model_id = "CompVis/stable-diffusion-v1-4"
model_dir_path = "/content/drive/MyDrive/Research/Models/" + model_id
print(model_dir_path)

out_dir = f"/content/drive/MyDrive/Research/Results/{lora_dir_name}/"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
print(out_dir)

if (os.path.exists(lora_path) or
  os.path.exists(model_dir_path) or
  os.path.exists(out_dir)):
  print("All directories exist")
else:
  from google.colab import runtime
  runtime.unassign()
  raise Exception("Directory not found")

In [ ]:
seed_num = 50
prompt = "black background"
lora_id = "rsd"
# prompt_lora = lora_id + " " + prompt
prompt_lora = lora_id

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_dir_path, torch_dtype=torch.float16)
pipe.to("cuda")

original_unet = pipe.unet

In [ ]:
def generate_and_save(pipe, prompt, seed, out_dir, suffix):
  image = pipe(
      prompt,
      generator=torch.Generator("cuda").manual_seed(seed)
      ).images[0]
  image.save(out_dir + f"seed{seed}_" + prompt.replace(" ", "_") + f"_{suffix}.png")

In [ ]:
# for seed in range(seed_num):
#   print(f"seed: {seed} / {seed_num - 1}")

#   generate_and_save(pipe, prompt, seed, out_dir, "ori")
#   generate_and_save(pipe, prompt_lora, seed, out_dir, "ori")

In [ ]:
lora_unet = PeftModel.from_pretrained(original_unet, lora_path, adapter_name=lora_id)
pipe.unet = lora_unet
pipe.to("cuda")

In [ ]:
# for seed in range(seed_num):
for seed in range(10, 50):
  print(f"seed: {seed} / {seed_num - 1}")

  # generate_and_save(pipe, prompt, seed, out_dir, "lora")
  generate_and_save(pipe, prompt_lora, seed, out_dir, "lora")

In [ ]:
from google.colab import runtime
runtime.unassign()